<a href="https://colab.research.google.com/github/aquila00/MovieIt/blob/main/MovieIt_item_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Sistema de Filtragem colaborativa - User Based

#Pré processamento

##Bibliotecas

In [ ]:
# Gerais
import pandas as pd
import numpy as np
import math

# Para separação de treino-teste
from sklearn.model_selection import train_test_split

# Para o RMSE
from sklearn.metrics import mean_squared_error

# Plot
import matplotlib.pyplot as plt
from pylab import title, xlabel, ylabel

# Google drive / Colab
from google.colab import drive

In [ ]:
# Acessando os arquivos no GoogleDrive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


##Database

### Ratings

In [ ]:
# 1 milhão
path = '/content/gdrive/MyDrive/movieit/ml-1m'
db_rating = pd.read_csv(f'{path}/ratings.dat', names=['userid', 'movieid', 'rating', 'timestamp'], usecols=['userid', 'movieid', 'rating'], skiprows=1, header=None, sep='::')

/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


### Gêneros

In [ ]:
# gêneros
db_movie = pd.read_csv(f'{path}/movies.dat', names=['movieid', 'movietitle', 'genres'], skiprows=1, header=None, sep='::')

In [ ]:
db_movie

,movieid,movietitle,genres
0,2,Jumanji (1995),Adventure|Children's|Fantasy
1,3,Grumpier Old Men (1995),Comedy|Romance
2,4,Waiting to Exhale (1995),Comedy|Drama
3,5,Father of the Bride Part II (1995),Comedy
4,6,Heat (1995),Action|Crime|Thriller
...,...,...,...
3877,3948,Meet the Parents (2000),Comedy
3878,3949,Requiem for a Dream (2000),Drama
3879,3950,Tigerland (2000),Drama
3880,3951,Two Family House (2000),Drama


##Exploratória

In [ ]:
db_rating

,userid,movieid,rating
0,1,661,3
1,1,914,3
2,1,3408,4
3,1,2355,5
4,1,1197,3
...,...,...,...
1000203,6040,1091,1
1000204,6040,1094,5
1000205,6040,562,5
1000206,6040,1096,4


In [ ]:
n_ratings = len(db_rating)
n_users = len(db_rating['userid'].unique())
n_movies_avaliados = len(db_rating['movieid'].unique())
esparsidade = round(1.0 - n_ratings/float(n_users * n_movies_avaliados), 3)

print(f'Total de ratings: {n_ratings}\n'\
      f'Total de usuários únicos: {n_users}\n'\
      f'Total de filmes: {n_movies_avaliados}\n'\
      f'Média de ratings/user: {round(n_ratings/n_users, 2)}\n'\
      f'Esparsidade da base é de: {(esparsidade*100)}%'\
      f'\n')

print(f'{db_rating.info()}')

Total de ratings: 1000208
Total de usuários únicos: 6040
Total de filmes: 3706
Média de ratings/user: 165.6
Esparsidade da base é de: 95.5%

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000208 entries, 0 to 1000207
Data columns (total 3 columns):
 #   Column   Non-Null Count    Dtype
---  ------   --------------    -----
 0   userid   1000208 non-null  int64
 1   movieid  1000208 non-null  int64
 2   rating   1000208 non-null  int64
dtypes: int64(3)
memory usage: 22.9 MB
None


## Criando os conjuntos de treino e teste

**Considerações:**

Os conjuntos de treino e de teste foram criados pelos ids dos usuários uma vez que o conjunto da votação de um usuário compõe a integridade de suas preferências. 

A separação dos dados sem considerar esta integridade geraria problemas de particionamento do perfil do usuário alterando o resultado da similaridade.

In [ ]:
# matriz dos ids únicos dos usuários para a refereência do particionamento treino e teste
print(db_rating.movieid.unique())

[ 661  914 3408 ... 2845 3607 2909]


In [ ]:
# Separando o conjunto de treino e de teste, deixando 10% dos usuários para futuros testes
X_train, X_test = train_test_split(db_rating.movieid.unique(), test_size=0.2, random_state=700)

In [ ]:
# retornando os valores de filmes e de ratings para completar a matriz de treino
X_train = db_rating.loc[db_rating['movieid'].isin(X_train)]

# retornando os valores de filmes e de ratings para completar a matriz de teste
X_test = db_rating.loc[db_rating['movieid'].isin(X_test)]

### X_train - Exploratória

In [ ]:
X_train.head()

,userid,movieid,rating
2,1,3408,4
3,1,2355,5
5,1,1287,5
6,1,2804,5
7,1,594,4


In [ ]:
X_train.count()

userid     788865
movieid    788865
rating     788865
dtype: int64

In [ ]:
X_train.head()

,userid,movieid,rating
2,1,3408,4
3,1,2355,5
5,1,1287,5
6,1,2804,5
7,1,594,4


In [ ]:
n_ratings = len(X_train)
n_users = len(X_train['userid'].unique())
n_movies_avaliados = len(X_train['movieid'].unique())
esparsidade = round(1.0 -n_ratings/float(n_users * n_movies_avaliados),3)

print(f'Total de ratings: {n_ratings}\n'\
      f'Total de usuários únicos: {n_users}\n'\
      f'Total de filmes: {n_movies_avaliados}\n'\
      f'Média de ratings/user: {round(n_ratings/n_users, 2)}\n'\
      f'Esparsidade da base é de: {(esparsidade*100)}%'\
      f'\n')

print(f'{db_rating.info()}')

Total de ratings: 788865
Total de usuários únicos: 6040
Total de filmes: 2964
Média de ratings/user: 130.61
Esparsidade da base é de: 95.6%

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000208 entries, 0 to 1000207
Data columns (total 3 columns):
 #   Column   Non-Null Count    Dtype
---  ------   --------------    -----
 0   userid   1000208 non-null  int64
 1   movieid  1000208 non-null  int64
 2   rating   1000208 non-null  int64
dtypes: int64(3)
memory usage: 22.9 MB
None


### X_test - Exploratória

In [ ]:
X_test.head()

,userid,movieid,rating
0,1,661,3
1,1,914,3
4,1,1197,3
11,1,2398,4
17,1,3105,5


In [ ]:
X_test.count()

userid     211343
movieid    211343
rating     211343
dtype: int64

In [ ]:
X_test.head()

,userid,movieid,rating
0,1,661,3
1,1,914,3
4,1,1197,3
11,1,2398,4
17,1,3105,5


In [ ]:
n_ratings = len(X_test)
n_users = len(X_test['userid'].unique())
n_movies_avaliados = len(X_test['movieid'].unique())
esparsidade = round(1.0 -n_ratings/float(n_users * n_movies_avaliados),3)

print(f'Total de ratings: {n_ratings}\n'\
      f'Total de usuários únicos: {n_users}\n'\
      f'Total de filmes: {n_movies_avaliados}\n'\
      f'Média de ratings/user: {round(n_ratings/n_users, 2)}\n'\
      f'Esparsidade da base é de: {(esparsidade*100)}%'\
      f'\n')

print(f'{db_rating.info()}')

Total de ratings: 211343
Total de usuários únicos: 6032
Total de filmes: 742
Média de ratings/user: 35.04
Esparsidade da base é de: 95.3%

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000208 entries, 0 to 1000207
Data columns (total 3 columns):
 #   Column   Non-Null Count    Dtype
---  ------   --------------    -----
 0   userid   1000208 non-null  int64
 1   movieid  1000208 non-null  int64
 2   rating   1000208 non-null  int64
dtypes: int64(3)
memory usage: 22.9 MB
None


# Recomendando filme para usuário único

## Filtrando a base

### Funções

In [ ]:
def instanciando_filme_unico(base, id_filmeunico):
    """O "uservip" é instanciado a partir de um um usuário real do conjunto X_test visando a otimização de tempo para a criação de um novo usuário."""

    # Uma amostra do X_test para otimização de tempo, utilizando o id informado na variável id_arbitrario
    filmeunico = base.loc[base['movieid'] == id_filmeunico]

    return filmeunico

In [ ]:
def criar_base_para_correlacao(X_treino, filme):
    """Seleciona todos os usuários da base de treino que tenham assistido pelo menos um filme em comum com o "usuario" e o próprio usuario, pivotados"""

    # Adiciona o usuario à tabela para que o número de colunas seja igual ao da base e para, futuramente, fazer a correlação
    base = pd.concat([X_treino, filme], ignore_index=True, axis='index')

    base_pivotada = base.pivot_table(index='userid', columns='movieid', values='rating')
   
    return base_pivotada

In [ ]:
def calculando_correlacao(base, id_filme, metodo='pearson'):
    """Calculando pearson por default"""
    print(base)
    
    corr = base.corrwith(base[id_filme], method=metodo)
    corr.drop(index=id_filme, inplace=True)
    corr = corr[corr>0]
    similaridades = corr.to_list()
    indexes = corr.index.to_list()    
    dicionario = dict(zip(indexes, similaridades))
    
    return dicionario

In [ ]:
def criar_base_para_recomendacao(X_treino, usuario, dicionario_vizinhos_pesos):
    """remove o usuário da base com todos filmes que os vizinhos do usuario assistiram"""

    base_user = X_treino.loc[~X_treino['userid'].isin(usuario['userid']) & X_treino['movieid'].isin(list(dicionario_vizinhos_pesos.keys()))]
    base_user['similaridade'] = [dicionario_vizinhos_pesos[X_treino['movieid'][index]] for index in base_user.index]

    return base_user

In [ ]:
#retorna o valor do coeficiente de pearson 
def valor_item(base, user):
    """Verifica a nota do item não consumido pelo usuário novo, com base nos usuários vizinhos"""

    base = base.loc[base['userid'] == user]

    resultado = 0
    divisor = 0
    
    for linha in base.index:
        valor = base.loc[base.index == linha, 'rating'].iloc[0]
        similaridade = base.loc[base.index == linha, 'similaridade'].iloc[0]
        
        resultado = resultado + valor * similaridade
        divisor = divisor + similaridade

    try:
        resultado = resultado / divisor     
    except ZeroDivisionError:
        resultado = 0

    return resultado

In [ ]:
def recomendacoes(base_treino, filmeid=1357, metodo_de_calculo='pearson', rmse=False):

    filme = instanciando_filme_unico(base_treino, filmeid)

    if rmse == True:
        filme_original = filme.copy()
        quantidade = math.floor(len(filme_original.index) * 0.1)
        drop_index = np.random.choice(filme.index, quantidade, replace=False)

        filme_original = filme_original[filme_original.index.isin(list(drop_index))]  
        filme = filme.drop(drop_index)

    db_corr = criar_base_para_correlacao(base_treino, filme)
    correlacao = calculando_correlacao(db_corr, filmeid, metodo_de_calculo)
    db_rec = criar_base_para_recomendacao(base_treino, filme, correlacao)
    #Cria o vetor contendo os dados do usuário novo usuário para teste alterar o valor 0
    nova_linha = [valor_item(db_rec, j) for j in db_rec.movieid.unique()]

    # Cria a variável que armazena os filmes a serem recomendados de acordo com os critérios de similaridade
    users_recomendados = [[resultado, filme] for resultado, filme in 
                          zip(nova_linha, db_rec.movieid.unique()) 
                          if resultado != 0]

    if rmse == True:
        extra = [[resultado[1], filmeid, resultado[0]] 
                 for resultado in users_recomendados 
                 if resultado[1] in filme_original.userid.unique()]

        extra = pd.DataFrame(extra, columns=filme_original.columns)

        return filme_original, extra
    else:
        return users_recomendados

### Recomendando

In [ ]:
id = 2858

In [ ]:
#ordena os filmes por ordem ascendente de notas e printa a nota na tela
filmes_recomendados = recomendacoes(X_train, filmeid=id)

movieid  2     5     6     7     10    11    13    14    15    16    ...  \
userid                                                               ...   
1         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
2         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
3         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
4         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
5         NaN   NaN   2.0   NaN   NaN   NaN   NaN   NaN   NaN   3.0  ...   
...       ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
6036      NaN   NaN   3.0   NaN   NaN   3.0   NaN   NaN   NaN   3.0  ...   
6037      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
6038      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
6039      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
6040      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   

movieid  39

/usr/local/lib/python3.8/dist-packages/numpy/lib/function_base.py:2683: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/usr/local/lib/python3.8/dist-packages/numpy/lib/function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
<ipython-input-22-857299bf0d80>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_user['similaridade'] = [dicionario_vizinhos_pesos[X_treino['movieid'][index]] for index in base_user.index]


In [ ]:
filmes_recomendados.sort(reverse=True)

In [ ]:
filmes_recomendados

[[4.906697328273252, 3324],
 [4.906155361553712, 446],
 [4.850389629031482, 1940],
 [4.80461851471212, 3902],
 [4.69352929938594, 1856],
 [4.680699822581582, 3306],
 [4.680094147941839, 567],
 [4.670664429912716, 2313],
 [4.668847859179639, 2574],
 [4.664011981633163, 215],
 [4.654570542482527, 1704],
 [4.643057388392296, 1488],
 [4.639761313486498, 2729],
 [4.638955189419225, 3181],
 [4.628222849552969, 1192],
 [4.620185453584355, 2467],
 [4.619940849001458, 152],
 [4.613578510927582, 3251],
 [4.6131822723792535, 2155],
 [4.607148790817002, 764],
 [4.60557379248111, 1371],
 [4.5901270691964315, 2694],
 [4.587534656389119, 2477],
 [4.57685178540592, 248],
 [4.575085285837197, 2791],
 [4.5692138993017375, 3221],
 [4.56722188949008, 2287],
 [4.565182979879042, 1702],
 [4.560244922847246, 3437],
 [4.549537096534865, 1349],
 [4.5489149396598485, 3347],
 [4.531102649751569, 121],
 [4.530181314477779, 521],
 [4.51791098267321, 1682],
 [4.515686087462051, 101],
 [4.514758190701181, 1188],
 [4

In [ ]:
print(f"O filme {db_movie.loc[db_movie['movieid']==id, 'movietitle'].values[0]}, "\
      #f"dos gêneros {db_movie.loc[db_movie['movieid']==id, 'generos'].values[0]}, "\
      f"é recomendado para os usuários:")
for i in range(20):
    print(f"{filmes_recomendados[i][1]} - Nota: {round(filmes_recomendados[i][0], 2)}")

O filme American Beauty (1999), é recomendado para os usuários:
3324 - Nota: 4.91
446 - Nota: 4.91
1940 - Nota: 4.85
3902 - Nota: 4.8
1856 - Nota: 4.69
3306 - Nota: 4.68
567 - Nota: 4.68
2313 - Nota: 4.67
2574 - Nota: 4.67
215 - Nota: 4.66
1704 - Nota: 4.65
1488 - Nota: 4.64
2729 - Nota: 4.64
3181 - Nota: 4.64
1192 - Nota: 4.63
2467 - Nota: 4.62
152 - Nota: 4.62
3251 - Nota: 4.61
2155 - Nota: 4.61
764 - Nota: 4.61


# Medidas de avaliação

In [ ]:
filmes = np.random.choice(X_train.movieid.unique(), 3, replace=False)

In [ ]:
for filme_temp in filmes:
    filme_original, user_predito = recomendacoes(X_train, filmeid=filme_temp, rmse=True, metodo_de_calculo='spearman')

    for userid in filme_original.userid:
        if userid not in user_predito.userid.unique():
            user_predito = pd.concat([user_predito, pd.DataFrame([[userid, filme_temp, 0]], columns=user_predito.columns)], ignore_index=True, axis='index')

    filme_original = filme_original.sort_values(by=['userid'])
    user_predito = user_predito.sort_values(by=['userid'])

    plt.plot(filme_original.userid, filme_original.rating)
    plt.plot(user_predito.userid, user_predito.rating)
    
    title(f"Notas para o filme {filme_temp}")
    xlabel("User")
    ylabel("Notas")
    plt.legend(['original', 'predito'])

    rms = mean_squared_error(filme_original.rating, user_predito.rating, squared=False)

    print('RMSE:')
    print(rms)
    plt.show()

movieid  2     5     6     7     10    11    13    14    15    16    ...  \
userid                                                               ...   
1         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
2         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
3         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
4         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
5         NaN   NaN   2.0   NaN   NaN   NaN   NaN   NaN   NaN   3.0  ...   
...       ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
6036      NaN   NaN   3.0   NaN   NaN   3.0   NaN   NaN   NaN   3.0  ...   
6037      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
6038      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
6039      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
6040      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   

movieid  39

/usr/local/lib/python3.8/dist-packages/scipy/stats/stats.py:4484: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(SpearmanRConstantInputWarning())
<ipython-input-22-857299bf0d80>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_user['similaridade'] = [dicionario_vizinhos_pesos[X_treino['movieid'][index]] for index in base_user.index]
